In [ ]:
#-----------------------------------------Aitzaz Ahmad -- 18I-0589-------------------------------------------------
#-----------------------------------------Muhammad Athar -- 18I-0692---------------------------------------------


import csv
import copy
from random import shuffle
from numpy import concatenate
from numpy import random
from numpy.random import randint

students_in_each_course = {}
courses_of_each_student = {}
courses_dict = {}
student_names = []
teacher_names = []
classrooms = []
dates = []
days = []
slots = ['09:00am - 12:00am', '02:00pm - 05:00pm']


##This function will load data from studentCourse file and will make two dictionaries from that data
## students_in_each_course will have structure like this {course_code: student_names list}
## courses_of_each_student will have structure like this {student_name: course code list}

def load_student_courses(filename):
    with open(filename) as csv_file:
        
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count > 0:
                if row[1] not in courses_of_each_student.keys() or row[2] not in courses_of_each_student[row[1]]:
                    students_in_each_course.setdefault(row[2], []).append(row[1])
                    courses_of_each_student.setdefault(row[1], []).append(row[2])
            line_count += 1
        
'''
        print("Length of our dict student in each course is ", len(students_in_each_course))
        for key, value in students_in_each_course.items():
            print("No of students enrolled in ", key, " are: ", len(value))
            
        print("Length of our dict courses of each student is ", len(courses_of_each_student))
        for key, value in courses_of_each_student.items():
            print(key, ": ", value)
'''


##this function will read from courses file and will make a dictionary in following format
## courses_dict = {course_code: course_name}

def load_courses_names(filename):
    with open(filename) as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=",")
        line_count = 0
        for row in csv_reader:
            courses_dict[row[0]] = row[1]
            line_count += 1

        '''    
        print("length of dict: ", len(courses_dict))
        for key, value in courses_dict.items():
            print(key, ": ", value)
        '''
        
##This function will read and store the names of student in a list
def load_student_names(filename):
    with open(filename) as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=",")
        for row in csv_reader:
            student_names.append(row[0])

        '''
        print("Len of std names is ", len(student_names))
        for i in student_names:
            print(i)
        '''
        
        
##This function will read and store the names of teachers in a list which will later be used for selection of invigilators
def load_teacher_names(filename):
    with open(filename) as csv_file:

        csv_reader = csv.reader(csv_file)
        for row in csv_reader:
            if row not in teacher_names and len(row) > 0:
                teacher_names.append(row[0])

        '''
        print("Len of tch names is ", len(teacher_names))
        for i in teacher_names:
            print(i)
        '''
        

##This function will be used for storing the available rooms in a list so that venue of exams could be decided
def load_classrooms(filename):
    with open('classrooms.csv') as csv_file:

        csv_reader = csv.reader(csv_file)
        line_count = 0
        for row in csv_reader:
            classrooms.append(row[0])
            line_count += 1
        
        '''
        print("Total avaialble classrooms are ", len(classrooms))
        for i in classrooms:
            print(i)
        '''
        
        
##This function will be used for storing the dates and days in seperate lists with date at ith index referring to day at ith index
def load_dates_and_days(filename):
    with open(filename) as csv_file:
        
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count > 0:
                dates.append(row[0])
                days.append(row[1])
            line_count += 1
        
        '''
        print("Length of our calender is ", len(dates), "==", len(days))
        for i in range(len(dates)):
            print(dates[i], "\t", days[i])
        '''
        
        
##This function will be used to generate the intial population which will be generated by selecting the data randomly
##from the above defined data structures. This function will create "papulation_size" no of chromosomes which will be 
##2d lists enclosed in one 3d list called population. This 3d list will behave as our population throughout the algorithm
def generate_population(population_size):
    population = [[[]]]                                    #This will include multiple samples of timetable and behave as our population
    chromosome = [[]]                                      #This will store one sample of timetable and at the end the best one will be shown among population
    individual = []                                        #This will store the information about one course in the exam schedule
    population.clear()
    chromosome.clear()
    individual.clear()

    ##Generating initial papulation by selecting the concerned data randomly
    for i in range(population_size):                             
        for key, value in courses_dict.items():
            no_of_students_adjusted = int(0)
            students_enrolled = len(courses_dict[key])
            
            course_code = key
            course_name = value
            while True:
                calender_index = random.randint(0, len(days)) 
                exam_date = dates[calender_index]
                exam_day = days[calender_index]
                if exam_day != 'Saturday' and exam_day != 'Sunday':
                    break
                    
            exam_slot = slots[random.randint(0, len(slots))]
            
            ##Functionality to add an extra classroom incase the students enrolled are more than room limit i.e 28
            while no_of_students_adjusted < students_enrolled:
                
                #Incase of more than 28 students, we need to pick another random room and invigilator. No need to change other parameters
                room = classrooms[random.randint(0, len(classrooms))]
                invigilator = teacher_names[random.randint(0, len(teacher_names))]
                if students_enrolled - no_of_students_adjusted <= 28:
                    students_range = students_enrolled
                    no_of_students_adjusted += students_enrolled
                else:
                    students_range = no_of_students_adjusted + 28
                    no_of_students_adjusted += 28
                individual.append(course_code)
                individual.append(course_name)
                individual.append(exam_date)
                individual.append(exam_day)
                individual.append(exam_slot)
                individual.append(invigilator)
                individual.append(room)
                individual.append(students_range)

                temp_individual = individual
                
                chromosome.append(copy.deepcopy(temp_individual))
                individual.clear()
        temp_chromosome = chromosome
        population.append(copy.deepcopy(temp_chromosome))
        chromosome.clear()
    
    return population
        
    
##This function will display population in an understandable manner
def display_population(population):
    counter = 1
    for chromosome in population:
        print("\t\t\t\t Chromosome#", counter, "\t\t\t")
        for individual in chromosome:
            for gene in individual:
                print(gene, end="  ")
            print()
        counter += 1
        print("\n\n")
        
##This function will display one chromosome unlike the above function which prints the whole population
##This function is mainly made to print the best solution obtained at the end of algorithm
def display_best_solution(chromosome):
    for individual in chromosome:
        for gene in individual:
            print(gene, end="  ")
        print()
    print("\n")
    

##This function will calculate the no of students having clashes in several courses. This number will also be added in fitness
##values and thus will impact the halting conditions of algorithm
def student_clashes(chromosome, printer=False):

    no_of_clashes = 0
    courses_of_this_student = []
    data_of_courses = [[]]
    clashed_courses = {}
    
    #loop through all the students 1 by 1 to know their courses
    for student in student_names:
        courses_of_this_student.clear()
        data_of_courses.clear()
        
        #store courses of each student in a list
        for course in courses_of_each_student[student]:
            courses_of_this_student.append(course)
            
        #checking clashes between any courses
        for individual in chromosome:
            if individual[0] in courses_of_this_student and len(data_of_courses) <= 0:
                data_of_courses.append(individual)
                courses_of_this_student.remove(individual[0])
            elif individual[0] in courses_of_this_student and len(data_of_courses) > 0:
                if individual[2] == data_of_courses[0][2] and individual[3] == data_of_courses[0][3] and individual[4] == data_of_courses[0][4]:
                    if individual[0] not in clashed_courses.keys() and data_of_courses[0][0] not in clashed_courses.values():
                        #if printer == True:
#                            print("******", student, " is having clash of ", individual[0], " with ", data_of_courses[0][0])
                        no_of_clashes += 1
                        clashed_courses[individual[0]] = data_of_courses[0][0]
    return no_of_clashes


##This function will check the no of exams on weekends so that they can be minimized
def exams_on_weekends(chromosomes):
    exams = 0
    for individual in chromosomes:
            if individual[3] == 'Saturday' or individual[3] == 'Sunday' :
                exams += 1
    return exams

##This function will return no of exams which have been accidently scheduled at the same room at the same time
def exam_on_same_room(chromosome):
    no_of_exams = 0
    for index in range(len(chromosome)):
        current_teacher = chromosome[index][5]
        current_date = chromosome[index][2]
        current_day = chromosome[index][3]
        current_time = chromosome[index][4]
        current_room = chromosome[index][6]
        
        for i in range(len(chromosome)):
            if i == index:
                continue
            if (chromosome[i][6] == current_room and chromosome[i][5] == current_teacher and chromosome[i][4] == current_time and chromosome[i][3] == current_day and chromosome[i][2] == current_date):
                    no_of_exams += 1
    return no_of_exams


##This function will return the no of invigilators who have been accidently scheduled at more than one place at the same time
def duplicate_invigilation(chromosome):
    
    invigilator_clashes = 0
    for index in range(len(chromosome)):
        current_teacher = chromosome[index][5]
        current_date = chromosome[index][2]
        current_day = chromosome[index][3]
        current_time = chromosome[index][4]
        
        for i in range(len(chromosome)):
            if i == index:
                continue
            if (chromosome[i][5] == current_teacher and chromosome[i][4] == current_time and chromosome[i][3] == current_day and chromosome[i][2] == current_date):
                    invigilator_clashes += 1
    return invigilator_clashes


##This function will return the no of invigilators who have been scheduled for concurrent invigilations
##if a teacher is invigilating from 9-12 and then 2-5 at the same day, it will be considered as concurrent invigilation
def concurrent_invigilation(chromosome):

    invigilator_clashes = 0
    for index in range(len(chromosome)):
        current_teacher = chromosome[index][5]
        current_date = chromosome[index][2]
        current_day = chromosome[index][3]
        current_time = chromosome[index][4]

        for i in range(len(chromosome)):
            if i == index:
                continue
            if current_time == '09:00am - 12:00am':
                if (chromosome[i][5] == current_teacher and chromosome[i][4] == '02:00pm - 05:00pm' and chromosome[i][3] == current_day and chromosome[i][2] == current_date):
                    invigilator_clashes += 1

                elif current_time == '02:00pm - 05:00pm':
                    if (chromosome[i][5] == current_teacher and chromosome[i][4] == '09:00am - 12:00am' and chromosome[i][3] == current_day and chromosome[i][2] == current_date):
                        invigilator_clashes += 1
    return invigilator_clashes


#This function will return the fitness of a provided chromosome which will actually be no of times the hard constraints have been violated
#The lower the fitness value, better the performance
def fitness_of_chromosome(chromosome, printer=False):
    
    exams = exams_on_weekends(chromosome)
    dup_room = exam_on_same_room(chromosome)
    #std_clash = student_clashes(chromosome, printer)
    std_clash = student_clashes(chromosome)
    dup_invg = duplicate_invigilation(chromosome)
    conc_invg = concurrent_invigilation(chromosome)
    
    if printer == True:
        print("\nFollowing hard constraints have been fulfilled: ")
        print("H1", end="\t")
        if std_clash == 0:
            print("H2", end="\t")
        if exams == 0:
            print("H3", end="\t")
        print("H4", end="\t")
        if dup_invg == 0:
            print("H5", end="\t")
        if conc_invg == 0:
            print("H6", end="\t")
            
        print("\tS1", end="\t")
        if std_clash == 0:
            print("S2", end="\t")
        print("S4\n")
        
        
        print("\nFollowing constraints have not been fulfilled yet\n")
        if std_clash != 0:
            print("H2", end="\t")
            print("A student cannot give more than 1 exam consecutively")
        if exams != 0:
            print("H3", end="\t")
        if dup_invg != 0:
            print("H5", end="\t")
        if conc_invg != 0:
            print("H6", end="\t")
        if std_clash != 0:
            print("S2\tS3\n")
        
    
        
    return exams + dup_room + std_clash + dup_invg + conc_invg


##This function will calculate the fitness value of entire population and store the fitness values of different chromosomes 
##in a list according to their indexes in the population and then return this 1d list
def calculate_fitness(population, population_size):
    

    fitnesses = [0] * population_size
    
    for counter in range(len(population)):
        chromosome = population[counter]
        fitnesses[counter] = fitness_of_chromosome(chromosome) 
           
    return fitnesses


##This function will select two chromsomes from the population as parent chromosomes which will later help in crossover and
##mutation. These parents will be selected according to roullete wheel selection
##Keep in mind that this function does not return the actual parents but it will send the indexes of those chromosomes which
##should be selected as the parent chromosomes
def parents_selection(population, our_fitness):

    ##as our fitness have inverse relation with the efficiency of algo so we will store the fitness values by subtraucting 
    ##them from their original sum so that minimum fitness value can have max chances in getting selected
    inverse_fitness = []
    parents_fitness = []
    parents_index= []
    parents_fitness.clear()
    parents_index.clear()
    
    first_parent_index = 0
    sec_parent_index = 0
    partial_sum = 0
    sum = 0
    inverse_sum = 0
    
    for i in our_fitness:
        sum += i
    for i in range(len(our_fitness)):
        inverse_fitness.append(sum - our_fitness[i])
        inverse_sum += inverse_fitness[i]
        
    ##selecting the first parent    
    num = random.randint(0, inverse_sum)   
    for i in range(len(inverse_fitness)):
        if partial_sum < num:
            partial_sum += inverse_fitness[i]
            if i == len(inverse_fitness)-1:
                first_parent_index = i
                parents_index.append(first_parent_index)
                parents_fitness.append(our_fitness[i])

        else:
            first_parent_index = i-1
            parents_index.append(first_parent_index)
            parents_fitness.append(our_fitness[i-1])
            break
    

    while True:
            ##selecting the second parent
        num = random.randint(0, sum)
        partial_sum = 0
        for i in range(len(inverse_fitness)):
            if partial_sum < num:
                partial_sum += inverse_fitness[i]
                if i == len(inverse_fitness)-1:
                    sec_parent_index = i
                    parents_index.append(sec_parent_index)
                    parents_fitness.append(our_fitness[i])
            else:
                sec_parent_index = i-1
                parents_index.append(sec_parent_index)
                parents_fitness.append(our_fitness[i-1])
                break
                
        ##if same chromsome is selected twice then repeat the loop to choose distinct 2nd parents
        if first_parent_index == sec_parent_index:
            if len(parents_fitness) > 0:
                parents_fitness.pop()
            if len(parents_index) > 0:
                parents_index.pop()
            continue
        else:
            break
    return parents_index
    

##This function will perform the crossover and then call the mutation function to do rest of the work
def apply_crossover(parents, crossover_probability, mutation_probability):
    
    crossovered_chromosomes = [[[]]]
    
    parent_1_org_fitness = fitness_of_chromosome(parents[0])
    parent_2_org_fitness = fitness_of_chromosome(parents[1])
    
    chromosome_1 = copy.deepcopy(parents[0])
    chromosome_2 = copy.deepcopy(parents[1])
    crossovered_chromosomes.clear()
 
    ##Appending the parent chromsomes because in case crossover could not be performed due to low crossover probability
    ##the offsprings will be same as the parent chromosomes
    crossovered_chromosomes.append(chromosome_1) 
    crossovered_chromosomes.append(chromosome_2)        
    
    
    if randint(0, 100) <= crossover_probability * 100:
        
        #Firslty doing crossover for the population and generating offsprings by merging the individuals of parents
        split_point = random.randint(0, min( len(parents[0]), len(parents[1]) ) )
    
        index = split_point
        while index < len(parents[1]):
            chromosome_1[index] = parents[1][index]
            index += 1

        index = split_point
        while index < len(parents[0]):
            chromosome_2[index] = parents[0][index]
            index += 1 

        #Now doing crossover for the schedule and generating offsprings by merging the genes of parents
        index_1 = 0
        index_2 = len(crossovered_chromosomes[1])-1
        while index_2 >= 0:
            crossovered_chromosomes[0][index_1][2] = crossovered_chromosomes[1][index_2][2]
            crossovered_chromosomes[0][index_1][3] = crossovered_chromosomes[1][index_2][3]
            crossovered_chromosomes[0][index_1][4] = crossovered_chromosomes[1][index_2][4]
            index_1 += 1
            index_2 -= 1

        index_1 = 0
        index_2 = len(crossovered_chromosomes[0])-1
        while index_2 >= 0:
            crossovered_chromosomes[1][index_1][2] = crossovered_chromosomes[0][index_2][2]
            crossovered_chromosomes[1][index_1][3] = crossovered_chromosomes[0][index_2][3]
            crossovered_chromosomes[1][index_1][4] = crossovered_chromosomes[0][index_2][4]
            index_1 += 1
            index_2 -= 1

        
    crossovered_chromosomes = apply_mutation(crossovered_chromosomes, mutation_probability)
        
    parent_1_new_fitness = fitness_of_chromosome(crossovered_chromosomes[0])
    parent_2_new_fitness = fitness_of_chromosome(crossovered_chromosomes[1])
    
    ##If the fitness after crossover and mutation is worse than undo this step
    if parent_1_new_fitness > parent_1_org_fitness:
        crossovered_chromosomes[0] = parents[0]
    if parent_2_new_fitness > parent_2_org_fitness:
        crossovered_chromosomes[1] = parents[1]
    
        
    return crossovered_chromosomes


##This function applies mutation on provided cromosomes with provided mutation probability
def apply_mutation(crossovered_chromosomes, mutation_probability):
    
    #applying mutation based on mutation probability
    if randint(0, 100) <= mutation_probability * 100:
        course = ""
        for chromosome in crossovered_chromosomes:        
            for individual in chromosome:
                temp_chromosome = copy.deepcopy(chromosome)
                fitness_org = fitness_of_chromosome(chromosome)
                
                ##if there are two entries for a course in the cromosome due to more no of students then we need to mutate solution
                ##in a way that both the papers are at same time
                if (individual[0] != course):
                    while True:
                        calender_index = random.randint(0, len(days)) 
                        exam_date = dates[calender_index]
                        exam_day = days[calender_index]
                        individual[2] = exam_date
                        individual[3] = exam_day
                        if exam_day != 'Saturday' and exam_day != 'Sunday':
                            break

                    exam_slot = slots[random.randint(0, len(slots))]
                    individual[4] = exam_slot
                    fitness_new = fitness_of_chromosome(chromosome)
                    if fitness_new > fitness_org:
                        chromosome = copy.deepcopy(temp_chromosome)
                course = individual[0]
                
    return crossovered_chromosomes

##This function woll return index of the cromosome with best fitness value in the population
def index_of_best_sample(population):
    min_fitness = 1000
    min_index = -1
    for i in range(len(population)):
        if fitness_of_chromosome(population[i]) < min_fitness:
            min_fitness = fitness_of_chromosome(population[i])
            min_index = i
    
    return min_index

##This function will return the best cromosome from the population based on the fitness value
def find_best_solution(population):
    
    min_fitness = 1000
    min_index = -1
    for i in range(len(population)):
        if fitness_of_chromosome(population[i]) < min_fitness:
            min_fitness = fitness_of_chromosome(population[i])
            min_index = i
            
    best_chrm = copy.deepcopy(population[min_index])
    
    return best_chrm


##This is the main driver function implementing the genetic algorithm
def runGA(population_size, max_generations, crossover_probability, mutation_probability):
    print("\n\nAbout to start genetic algorithm\n\n")
    print("We have to fullfill following constraints")
    print("H1: An exam will be scheduled for each course")
    print("H2: A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a time.")
    print("H3: Exam will not be held on weekends")
    print("H4: Each exam must be held between 9 am and 5 pm")
    print("H5: Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.")
    print("H6: A teacher cannot invigilate two exams in a row.")
    print("S1: All students and teachers shall be given a break on Friday from 1-2")
    print("S2: A student shall not give more than 1 exam consecutively")
    print("S3: If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam")
    print("S4: Two hours of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot so the faculty meetings shall be held in parts as they are now.")
    
    #generate initial population
    population = generate_population(population_size)
    
    print("Max no of generations: ", max_generations)
    
    best_solution = None
    best_fitness = -1
    best_index = -1
    parent_chromosomes = [[[]]]
    
    for loop_counter in range(max_generations):

        ## Calculate Fitness of initial population
        ## This function will return a 1D list containing fitness of each chromosome 
        population_fitness = calculate_fitness(population, len(population)) #--
    
        ## Select parent
        parents_indexes = parents_selection(population, population_fitness) #--
        parent_chromosomes.clear()
        for i in parents_indexes:
            parent_chromosomes.append(copy.deepcopy(population[i]))
            
        ## Apply crossover and mutation
        offsprings = apply_crossover(parent_chromosomes, crossover_probability, mutation_probability)

        ##calculate fitness of offsprings calculated through crossover and mutation and the parents selected earlier
        parents_fitness = calculate_fitness(parent_chromosomes, len(parent_chromosomes))
        offsprings_fitness = calculate_fitness(offsprings, len(offsprings))

        ##do the required modifications in the population based on the fitness value of offsprings produced
        for i in range(len(parent_chromosomes)):
            if offsprings_fitness[i] < parents_fitness[i]:
                population[parents_indexes[i]] = copy.deepcopy(offsprings[i])
        
        ##Best solution at this moment
        best_solution = find_best_solution(population)
        result = fitness_of_chromosome(best_solution)
        
        if loop_counter % 10 == 0:
            print("Iteration # : ", loop_counter)
            result = fitness_of_chromosome(best_solution, True)
            print("Best fitness = ", result)
        
        result = fitness_of_chromosome(best_solution)
        if result == 0:
            print("\n\n\nSolution found")
            print("Iteration # : ", loop_counter)
            fitness_of_chromosome(best_solution, True)
            print("Best fitness = ", result)
            break
                
    display_best_solution(best_solution)
    
def main():
    print("Welcome to main function")
    load_student_courses('./original/studentCourse.csv')
    load_courses_names('./original/courses.csv')
    load_student_names('./original/studentNames.csv')
    load_teacher_names('./original/teachers.csv')
    load_classrooms('classrooms.csv')
    load_dates_and_days('dates_day.csv')
    
    population_size = 10#random.randint(5, 10)    
    max_generations = 4000#random.randint(100, 1000)
    crossover_probability = round(random.uniform(low=0.3, high=1.0), 1)
    mutation_probability = round(random.uniform(low=0.1, high=0.5), 1)
    
    print ('\n\n--- Generated Parameters -----')
    print ('Population size......: {}' .format(population_size))
    print ('Number of generations: {}'.format(max_generations))
    print ('Crossover probability: {}'.format(crossover_probability))
    print ('Mutation probability.: {}'.format(mutation_probability))
    runGA(population_size, max_generations, crossover_probability, mutation_probability)
    
if __name__ == "__main__":
    main()

Welcome to main function


--- Generated Parameters -----
Population size......: 10
Number of generations: 4000
Crossover probability: 0.8
Mutation probability.: 0.3


About to start genetic algorithm


We have to fullfill following constraints
H1: An exam will be scheduled for each course
H2: A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a time.
H3: Exam will not be held on weekends
H4: Each exam must be held between 9 am and 5 pm
H5: Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.
H6: A teacher cannot invigilate two exams in a row.
S1: All students and teachers shall be given a break on Friday from 1-2
S2: A student shall not give more than 1 exam consecutively
S3: If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam
S4: Two hours of break in the week such that at least half the faculty is free in one slot and t

Iteration # :  320

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  330

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  340

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  350

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  360

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Followi

Iteration # :  690

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  700

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  710

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  720

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  730

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Followi

Iteration # :  1060

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  1070

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  1080

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  1090

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  1100

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Fo

Iteration # :  1430

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  1440

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  1450

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  1460

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Following constraints have not been fulfilled yet

H2	A student cannot give more than 1 exam consecutively
S2	S3

Best fitness =  3
Iteration # :  1470

Following hard constraints have been fulfilled: 
H1	H3	H4	H5	H6		S1	S4


Fo